In [152]:
import yfinance as yf 
import time
import datetime
import pandas as pd
import sqlite3

In [166]:
month_symbol_mapper = {
    'jul':'N',
    'aug':'Q',
    'sep':'U',
    'oct':'V',
    'nov':'X',
    'dec':'Z',
    'jan':'F',
    'feb':'G',
    'mar':'H',
    'apr':'J',
    'may':'K',
    'jun':'M',
}

def oil_futures_ticker_generator(year='22',month='jan'):
    ticker =  f"CL{month_symbol_mapper[month]}{year}.NYM"
    print(ticker)
    return ticker

def get_yahoo_data(ticker="CLN22.NYM",start=None,end=None):
    yahoo_ticker_interface = yf.Ticker(ticker)
    if start is None or end is None:        
        price_data = yahoo_ticker_interface.history(period='max')
    else:
        price_data = yahoo_ticker_interface.history(start=start,end=end)
    price_data.drop(labels=['Dividends','Stock Splits'], axis=1, inplace=True)
    price_data['Ticker']=ticker
    return price_data

def get_front_month():
    month_string = datetime.datetime(datetime.date.today().year,datetime.date.today().month+2,1).strftime('%b').lower()
    month_number = datetime.datetime(datetime.date.today().year,datetime.date.today().month+2,1).strftime('%-m')
    return {'month_string': month_string, 'month_number':month_number}

def get_back_month():
    month_string = datetime.datetime(datetime.date.today().year,datetime.date.today().month+3,1).strftime('%b').lower()
    month_number = datetime.datetime(datetime.date.today().year,datetime.date.today().month+3,1).strftime('%-m')
    return {'month_string': month_string, 'month_number':month_number}

def get_current_cycle():
    cycle_start = datetime.datetime(datetime.date.today().year,datetime.date.today().month,1)
    today = datetime.date.today()
    return {'cycle_start':cycle_start, 'today':today}


In [167]:
front_month = get_front_month()
back_month = get_back_month()
front_month_ticker = oil_futures_ticker_generator(month= front_month['month_string'])
back_month_ticker = oil_futures_ticker_generator(month= back_month['month_string'])
current_cycle = get_current_cycle()
front_month_data = get_yahoo_data(ticker=front_month_ticker,start=current_cycle['cycle_start'],end=current_cycle['today'])
back_month_data = get_yahoo_data(ticker=back_month_ticker,start=current_cycle['cycle_start'],end=current_cycle['today'])


CLN22.NYM
CLQ22.NYM


In [168]:
front_month_data.head()


,Open,High,Low,Close,Volume,Ticker
Date,,,,,,
2022-05-02,102.150002,104.190002,98.870003,103.419998,89425,CLN22.NYM
2022-05-03,103.190002,104.059998,100.580002,100.900002,97505,CLN22.NYM
2022-05-04,101.989998,107.000000,101.410004,106.220001,92053,CLN22.NYM
2022-05-05,106.070000,109.769997,104.949997,106.739998,97440,CLN22.NYM
2022-05-06,107.070000,109.529999,105.720001,108.330002,119325,CLN22.NYM


In [169]:
back_month_data.head()

,Open,High,Low,Close,Volume,Ticker
Date,,,,,,
2022-05-02,100.260002,102.110001,97.120003,101.379997,50035,CLQ22.NYM
2022-05-03,101.190002,102.040001,98.839996,99.220001,57321,CLQ22.NYM
2022-05-04,100.269997,105.120003,99.750000,104.419998,51765,CLQ22.NYM
2022-05-05,104.180000,107.800003,103.150002,104.860001,61801,CLQ22.NYM
2022-05-06,105.010002,107.529999,103.839996,106.480003,53139,CLQ22.NYM


In [170]:
merged = pd.merge(front_month_data, back_month_data, left_index=True, right_index=True, how='inner', suffixes=('_front','_back'))
merged['Time_Spread']= merged['Close_back']-merged['Close_front']
merged

,Open_front,High_front,Low_front,Close_front,Volume_front,Ticker_front,Open_back,High_back,Low_back,Close_back,Volume_back,Ticker_back,Time_Spread
Date,,,,,,,,,,,,,
2022-05-02,102.150002,104.190002,98.870003,103.419998,89425,CLN22.NYM,100.260002,102.110001,97.120003,101.379997,50035,CLQ22.NYM,-2.040001
2022-05-03,103.190002,104.059998,100.580002,100.900002,97505,CLN22.NYM,101.190002,102.040001,98.839996,99.220001,57321,CLQ22.NYM,-1.680000
2022-05-04,101.989998,107.000000,101.410004,106.220001,92053,CLN22.NYM,100.269997,105.120003,99.750000,104.419998,51765,CLQ22.NYM,-1.800003
2022-05-05,106.070000,109.769997,104.949997,106.739998,97440,CLN22.NYM,104.180000,107.800003,103.150002,104.860001,61801,CLQ22.NYM,-1.879997
2022-05-06,107.070000,109.529999,105.720001,108.330002,119325,CLN22.NYM,105.010002,107.529999,103.839996,106.480003,53139,CLQ22.NYM,-1.849998
2022-05-09,108.970001,109.000000,100.809998,101.769997,148435,CLN22.NYM,107.070000,107.129997,99.160004,100.099998,62872,CLQ22.NYM,-1.669998
2022-05-10,100.959999,102.779999,97.610001,98.449997,156436,CLN22.NYM,99.849998,101.010002,96.150002,96.870003,66358,CLQ22.NYM,-1.579994
2022-05-11,97.739998,104.739998,96.930000,104.029999,179721,CLN22.NYM,96.250000,102.809998,95.470001,102.199997,76907,CLQ22.NYM,-1.830002
2022-05-12,103.940002,105.550003,101.110001,104.400002,179610,CLN22.NYM,102.400002,103.480003,99.360001,102.419998,70925,CLQ22.NYM,-1.980003


In [171]:
oil = yf.Ticker("CLN22.NYM")
#oil.history(start=datetime.datetime(datetime.date.today().year,datetime.date.today().month-2,1))

oil = get_yahoo_data(ticker="CLN22.NYM", start=datetime.datetime(datetime.date.today().year,datetime.date.today().month-2,1), end=datetime.datetime.now())
oil
# ticker = oil_futures_ticker_generator(year='22',month='jul')
# hist['ticker']=ticker
# hist.head()
# hist.index

,Open,High,Low,Close,Volume,Ticker
Date,,,,,,
2022-03-01,88.820000,94.650002,88.559998,92.639999,58813,CLN22.NYM
2022-03-02,94.900002,99.160004,93.279999,97.580002,92251,CLN22.NYM
2022-03-03,97.820000,102.349998,94.790001,96.779999,60183,CLN22.NYM
2022-03-04,97.059998,104.440002,97.000000,104.180000,57267,CLN22.NYM
2022-03-07,109.779999,116.430000,103.330002,107.720001,62041,CLN22.NYM
2022-03-08,108.629997,115.040001,105.910004,110.830002,73501,CLN22.NYM
2022-03-09,112.180000,113.669998,91.940002,97.260002,84957,CLN22.NYM
2022-03-10,98.080002,102.639999,95.570000,96.730003,87492,CLN22.NYM
2022-03-11,97.190002,100.610001,95.220001,100.480003,54173,CLN22.NYM


In [165]:
datetime.datetime(datetime.date.today().year,datetime.date.today().month,1)
#datetime.datetime(datetime.date.today().year,datetime.date.today().month+3,1).strftime('%h').lower()
#datetime.date.today().strftime('%b').lower()
# datetime.datetime.now()
#datetime.datetime(datetime.date.today().year,datetime.date.today().month+2,1).strftime('%-m')

datetime.datetime(2022, 5, 1, 0, 0)

## Database connectivity

In [153]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [154]:
conn = create_connection('futures.db')

In [183]:
def create_entry(conn, row):
    """
    Create a new futures price entry in the database's OIL_FUTURES_YAHOO table
    :param conn:
    :param row:
    :return: void
    """
    sql = '''INSERT INTO OIL_FUTURES_YAHOO(TICKER, DATE, OPEN, HIGH, LOW, CLOSE, VOLUME)
            VALUES(?,?,?,?,?,?,?)'''
    try:
        cur = conn.cursor()
        cur.execute(sql,row)
        conn.commit()
    except Exception as e:
        print(e)

In [186]:
for index,row in back_month_data.iterrows():
    print((row[5],index.date(),row[0],row[1],row[2],row[3],row[4]))
    row = (row[5],index.date(),row[0],row[1],row[2],row[3],row[4])
    create_entry(conn, row)


('CLQ22.NYM', datetime.date(2022, 5, 2), 100.26000213623047, 102.11000061035156, 97.12000274658203, 101.37999725341797, 50035)
('CLQ22.NYM', datetime.date(2022, 5, 3), 101.19000244140625, 102.04000091552734, 98.83999633789062, 99.22000122070312, 57321)
('CLQ22.NYM', datetime.date(2022, 5, 4), 100.2699966430664, 105.12000274658203, 99.75, 104.41999816894531, 51765)
('CLQ22.NYM', datetime.date(2022, 5, 5), 104.18000030517578, 107.80000305175781, 103.1500015258789, 104.86000061035156, 61801)
('CLQ22.NYM', datetime.date(2022, 5, 6), 105.01000213623047, 107.52999877929688, 103.83999633789062, 106.4800033569336, 53139)
('CLQ22.NYM', datetime.date(2022, 5, 9), 107.06999969482422, 107.12999725341797, 99.16000366210938, 100.0999984741211, 62872)
('CLQ22.NYM', datetime.date(2022, 5, 10), 99.8499984741211, 101.01000213623047, 96.1500015258789, 96.87000274658203, 66358)
('CLQ22.NYM', datetime.date(2022, 5, 11), 96.25, 102.80999755859375, 95.47000122070312, 102.19999694824219, 76907)
('CLQ22.NYM', 

## testing the library

In [187]:
import yf_interface